<a href="https://colab.research.google.com/github/kevinshiroya123/Movie-Recommendation-system/blob/main/movie_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!kaggle datasets download -d asaniczka/tmdb-movies-dataset-2023-930k-movies -p /kaggle
!unzip /kaggle/tmdb-movies-dataset-2023-930k-movies.zip -d /kaggle
import os
print(os.listdir("/kaggle"))


Dataset URL: https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies
License(s): ODC Attribution License (ODC-By)
 97% 200M/206M [00:01<00:00, 177MB/s]
100% 206M/206M [00:01<00:00, 157MB/s]
Archive:  /kaggle/tmdb-movies-dataset-2023-930k-movies.zip
  inflating: /kaggle/TMDB_movie_dataset_v11.csv  
['tmdb-movies-dataset-2023-930k-movies.zip', 'TMDB_movie_dataset_v11.csv']


In [ ]:
import pandas as pd

file_path = "/kaggle/TMDB_movie_dataset_v11.csv"
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(df.head())



       id            title  vote_average  vote_count    status release_date  \
0   27205        Inception         8.364       34495  Released   2010-07-15   
1  157336     Interstellar         8.417       32571  Released   2014-11-05   
2     155  The Dark Knight         8.512       30619  Released   2008-07-16   
3   19995           Avatar         7.573       29815  Released   2009-12-15   
4   24428     The Avengers         7.710       29166  Released   2012-04-25   

      revenue  runtime  adult                     backdrop_path  ...  \
0   825532764      148  False  /8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg  ...   
1   701729206      169  False  /pbrkL804c8yAv3zBZR4QPEafpAR.jpg  ...   
2  1004558444      152  False  /nMKdUUepR0i5zn0y1T4CsSB5chy.jpg  ...   
3  2923706026      162  False  /vL5LR6WdxWPjLPFRLe133jXWsh5.jpg  ...   
4  1518815515      143  False  /9BBTo63ANSmhC4e6r62OJFuK2GL.jpg  ...   

    original_title                                           overview  \
0        Inception 

In [ ]:
import pandas as pd

# Load the dataset
file_path = "/kaggle/TMDB_movie_dataset_v11.csv"
df = pd.read_csv(file_path)

# Preprocess the data
df['overview'] = df['overview'].fillna('')
df['genres'] = df['genres'].fillna('')
df['keywords'] = df['keywords'].fillna('')

# Remove duplicates by keeping the row with the highest vote_count
df = df.sort_values(by='vote_count', ascending=False).drop_duplicates(subset='title', keep='first')

# Remove rows with missing titles
df = df.dropna(subset=['title']).reset_index(drop=True)

# Normalize popularity and vote count
df['normalized_popularity'] = (df['popularity'] - df['popularity'].min()) / (df['popularity'].max() - df['popularity'].min())
df['normalized_vote_count'] = (df['vote_count'] - df['vote_count'].min()) / (df['vote_count'].max() - df['vote_count'].min())

# Combine relevant text features for content-based recommendations
df['combined_features'] = (
    df['genres'] + " " +  # Genres
    df['keywords'] + " " +
    df['overview']  # Overview
)

# Save the preprocessed data to a new CSV file
preprocessed_file_path = "/kaggle/preprocessed_movies.csv"
df.to_csv(preprocessed_file_path, index=False)
print(f"Preprocessed data saved to {preprocessed_file_path}")


Preprocessed data saved to /kaggle/preprocessed_movies.csv


In [ ]:
!pip install faiss-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 13.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import faiss
import numpy as np

# Load the preprocessed dataset
preprocessed_file_path = "/kaggle/preprocessed_movies.csv"
df = pd.read_csv(preprocessed_file_path)

# Normalize popularity and vote count
df['normalized_popularity'] = (df['popularity'] - df['popularity'].min()) / (df['popularity'].max() - df['popularity'].min())
df['normalized_vote_count'] = (df['vote_count'] - df['vote_count'].min()) / (df['vote_count'].max() - df['vote_count'].min())

# Use TF-IDF Vectorizer to process the combined features column
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

# Convert TF-IDF matrix to dense format and ensure it's float32 for FAISS compatibility
dense_matrix = tfidf_matrix.toarray().astype(np.float32)

# Save the dense matrix in compressed format
np.savez_compressed("/kaggle/dense_matrix_compressed.npz", dense_matrix)

# Build a FAISS index (batch-wise addition for large datasets)
faiss_index = faiss.IndexFlatIP(dense_matrix.shape[1])  # Inner Product for cosine similarity

# Batch-wise addition to FAISS index
batch_size = 10000  # Adjust batch size based on available memory
for start_idx in range(0, dense_matrix.shape[0], batch_size):
    end_idx = min(start_idx + batch_size, dense_matrix.shape[0])
    faiss_index.add(dense_matrix[start_idx:end_idx])

# Serialize the FAISS index to a NumPy array
faiss_index_bytes = faiss.serialize_index(faiss_index)

# Save the serialized FAISS index in compressed format
np.savez_compressed("/kaggle/faiss_index_compressed.npz", faiss_index_bytes=faiss_index_bytes)

# Save the processed DataFrame
df.to_csv("/kaggle/processed_movies.csv", index=False)

print("Preprocessing complete. Data saved in optimized format.")


Preprocessing complete. Data saved in optimized format.


In [ ]:
dense_matrix = np.load("/kaggle/dense_matrix.npy")  # Load the dense matrix
print(f"Dense Matrix Shape: {dense_matrix.shape}")
print(f"FAISS Index Dimensionality: {faiss_index.d}")


Dense Matrix Shape: (992313, 5000)
FAISS Index Dimensionality: 5000


In [ ]:
import pandas as pd
import faiss
import numpy as np
import gzip
import shutil

# Global variables to store the loaded data
loaded_df = None
loaded_faiss_index = None
loaded_dense_matrix = None

def load_data():
    """
    Load the preprocessed dataset, FAISS index, and dense matrix into global variables.
    """
    global loaded_df, loaded_faiss_index, loaded_dense_matrix

    # Load the dataset
    if loaded_df is None:
        print("Loading dataset...")
        loaded_df = pd.read_csv("/kaggle/processed_movies.csv")
    else:
        print("Dataset already loaded.")

    # Load and decompress the FAISS index
    if loaded_faiss_index is None:
        print("Decompressing and loading FAISS index...")
        try:
            compressed_faiss_file = np.load("/kaggle/faiss_index_compressed.npz")
            faiss_index_bytes = compressed_faiss_file['faiss_index_bytes']
            loaded_faiss_index = faiss.deserialize_index(faiss_index_bytes)
        except Exception as e:
            print(f"Error loading FAISS index: {e}")
    else:
        print("FAISS index already loaded.")

    # Load the dense matrix
    if loaded_dense_matrix is None:
        print("Loading dense matrix...")
        try:
            dense_matrix_file = np.load("/kaggle/dense_matrix_compressed.npz")
            loaded_dense_matrix = dense_matrix_file['arr_0']
        except Exception as e:
            print(f"Error loading dense matrix: {e}")
    else:
        print("Dense matrix already loaded.")

    return loaded_df, loaded_faiss_index, loaded_dense_matrix

# Call the function once at the start of the session
df, faiss_index, dense_matrix = load_data()


Loading dataset...
Decompressing and loading FAISS index...
Loading dense matrix...


In [ ]:
def recommend_movies(title, num_recommendations, min_vote_count, genre_weight, popularity_weight, vote_count_weight, popularity_threshold=0.8):
    """
    Recommend movies similar to the given title based on the FAISS index and preprocessed data.
    Dynamically combines genres from all movies that match the input title and uses them for recommendations.
    """
    global loaded_df, loaded_faiss_index, loaded_dense_matrix

    # Use already loaded global variables
    df = loaded_df
    faiss_index = loaded_faiss_index
    dense_matrix = loaded_dense_matrix

    # Normalize title for case-insensitive matching
    title = title.lower()

    # Find movies matching the input title
    matching_movies = df[df['title'].str.lower().str.contains(title)]
    matching_indices = matching_movies.index

    # Check if there are matching movies
    if len(matching_indices) == 0:
        return f"No movies found containing '{title}' in their title!"

    # Dynamically combine genres of all matching movies
    combined_genres = "|".join(matching_movies['genres'].dropna().unique())

    # Filter the dataset to reduce search space based on popularity
    filtered_df = df[df['popularity'] > df['popularity'].quantile(popularity_threshold)]  # Top 20% by popularity
    filtered_indices = filtered_df.index

    # Extract filtered vectors from the dense matrix
    filtered_dense_matrix = dense_matrix[filtered_indices]

    # Create a temporary FAISS index for the filtered dataset
    temp_faiss_index = faiss.IndexFlatIP(filtered_dense_matrix.shape[1])
    temp_faiss_index.add(filtered_dense_matrix)

    # Query vector from the dense matrix (use the first match for the query)
    query_vector = dense_matrix[matching_indices[0]].reshape(1, -1)

    # Perform FAISS search on the filtered index
    distances, indices = temp_faiss_index.search(query_vector, num_recommendations + 10)  # Search for slightly more movies

    # Map filtered indices back to the original dataset
    sim_indices = filtered_indices[indices.flatten()]

    # Scale distances by genre weight
    distances = distances.flatten() * genre_weight

    # Add scaled weights for popularity and vote count
    cumulative_similarity = distances
    cumulative_similarity += popularity_weight * df.loc[sim_indices, 'normalized_popularity'].values
    cumulative_similarity += vote_count_weight * df.loc[sim_indices, 'normalized_vote_count'].values

    # Sort by similarity and select top recommendations
    sorted_indices = np.argsort(cumulative_similarity)[::-1]
    top_sim_indices = sim_indices[sorted_indices[:num_recommendations]]

    # Filter recommended movies by combined genres and vote count
    recommended_movies = df.loc[top_sim_indices, ['title', 'genres', 'popularity', 'vote_count']]
    recommended_movies = recommended_movies[recommended_movies['genres'].str.contains(combined_genres, na=False)]
    recommended_movies = recommended_movies[recommended_movies['vote_count'] > min_vote_count]

    return recommended_movies


In [ ]:
# Example usage
movie_title = "Batman"  # Replace with a valid movie title
print(f"Input movie: {movie_title}")

# Get recommendations with dynamically combined genres
recommendations = recommend_movies(
    title=movie_title,
    num_recommendations=100,
    min_vote_count=100,
    genre_weight=2,
    popularity_weight=1.5,
    vote_count_weight=1,
    popularity_threshold=0.8  # Top 20% by popularity
)

# Display recommendations
if isinstance(recommendations, pd.DataFrame):
    print(f"Movies recommended for '{movie_title}':")
    print(recommendations)
else:
    print(recommendations)


Input movie: Batman
Movies recommended for 'Batman':
                                                   title  \
41                                         Batman Begins   
2                                        The Dark Knight   
25                                 The Dark Knight Rises   
57                               Spider-Man: No Way Home   
68                                            Spider-Man   
484                                               Batman   
132                                         Spider-Man 2   
85                                            Deadpool 2   
901                                       Batman & Robin   
228                                             Kick-Ass   
363                                           The Batman   
636                                       Batman Returns   
573                         Teenage Mutant Ninja Turtles   
732                                                Blade   
325                         Zack Snyder's Justi